In [ ]:
!pip install transformers faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load dataset
df = pd.read_csv("/content/Indian_Food_Dataset_With_Recipes.csv")
df = df.dropna(subset=["Cleaned-Ingredients", "TranslatedRecipeName", "FullRecipe"])

# Concatenate relevant fields for embedding
df["retrieval_text"] = (
    df["TranslatedRecipeName"] + " | " +
    df["Cleaned-Ingredients"] + " | " +
    df["FullRecipe"]
)

# Create embeddings using SentenceTransformer
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embed_model.encode(df["retrieval_text"].tolist(), show_progress_bar=True)

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Load LLM from Hugging Face
model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)

# Function to search and generate answer
def get_recipe_suggestions(user_query):
    user_embed = embed_model.encode([user_query])
    _, indices = index.search(user_embed, k=3)

    # Collect relevant recipes
    recipes = df.iloc[indices[0]][["TranslatedRecipeName", "TotalTimeInMins", "TranslatedIngredients", "TranslatedInstructions"]]

    # Build context string
    context = ""
    for idx, row in recipes.iterrows():
        context += f"🍽️ Recipe: {row['TranslatedRecipeName']}\n"
        context += f"⏱️ Time: {row['TotalTimeInMins']} mins\n"
        context += f"🧂 Ingredients: {row['TranslatedIngredients']}\n"
        context += f"👨‍🍳 Instructions: {row['TranslatedInstructions']}\n\n"

    # Prompt to reduce hallucination
    prompt = f"Based on the recipes below, list exactly 3 recipes matching the query '{user_query}' with their cooking time, ingredients, and instructions:\n\n{context}\n\nOnly use the given data and avoid adding any extra recipes or facts."

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=500)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
query = input("Enter ingredients or dish you're looking for: ")
result = get_recipe_suggestions(query)
print(result)

Enter ingredients or dish you're looking for: chicken, eggs, onions, tomatoes, red chilli powder, salt, turmeric powder


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the recipes below, list exactly 3 recipes matching the query 'chicken, eggs, onions, tomatoes, red chilli powder, salt, turmeric powder' with their cooking time, ingredients, and instructions:

🍽️ Recipe:  No Onion No Garlic Cabbage Kofta Curry
⏱️ Time: 65 mins
🧂 Ingredients:  1 green chili - chopped, 1/2 tsp turmeric powder, 1 sprig of green coriander - cut, 1/2 tsp garam masala powder,2 cups cabbage - grated, 1/2 cup gram flour, 1/2 tsp red chilli powder, 1/2 tsp cumin powder, turmeric powder - A pinch, 1 to 2 bay leaves, 3 to 4 tbsp cream, 1/2 tsp cardamom - break it, make 3 tomatoes - puree, 1/2 tsp salt - as per taste, oil - as per use, 1/2 tsp hot Masala powder, 1-1 / 2 tbsp coriander powder, 1 tbsp ginger - paste, 1 tbsp red chilli powder, salt - according to taste, 2 tbsp coriander - chopped, 1 cup oil - as per use, 2 tbsp yogurt
👨‍🍳 Instructions: To make the Cabbage Kofta Curry Recipe, firstly tighten the cabbage and put it in a bowl.
Take out the water from it by pre